In [1]:
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numba import jit
from pathlib import Path
from skimage import draw,transform

In [ ]:
# # Parameters
# pixel_size = 1.6537 # nm
# pixel_size = 1.825 # nm
pixel_size = 2.4335 # nm
em_path = '/Users/lukasvandenheuvel/Documents/PhD/Data-temp/20241022_iDA-figures/clem-immunogold/raw/Matek2-cU/Image-0059.tif'
gold_locations_path = '/Users/lukasvandenheuvel/Documents/PhD/Data-temp/20241022_iDA-figures/clem-immunogold/raw/Matek2-cU/ig-detection-inset02/Image-0059-goldLocations.csv'
out_path = '/Users/lukasvandenheuvel/Documents/PhD/Data-temp/20241022_iDA-figures/clem-immunogold/raw/Matek2-cU/ig-detection-inset02'
num_closest_gold = 15 # The heatmap will show the sum of the distance to the num_closest_gold closest gold particles.
final_width = 2048 # Downscale the EM image and heatmap to this width. This will drastically reduce running time.

In [3]:
# @jit # Just-in-time accelleration: uncommenting this line may result in a faster performance (but for me it didn't...))
def distance(height,width,G,num_closest=10):
    # Calculates for each pixel the sum of the distances to the num_closest gold particles 
    D = np.zeros((height,width),dtype=np.int32)
    for y in range(height):
        for x in range(width):
            distance_array = np.sqrt((x - G[:,1].reshape(-1,1).T)**2 + (y - G[:,0].reshape(-1,1).T)**2)
            D[y,x] = np.sum(np.sort(distance_array)[0,:num_closest])
    return D

# Calculate distance to N closest gold particles

In [4]:
# Read data
gold_locations = pd.read_csv(gold_locations_path)
em = io.imread(em_path)

In [5]:
# Read data
gold_locations = pd.read_csv(gold_locations_path)
em = io.imread(em_path)
rescale_factor = final_width/em.shape[1]
em_rescaled = transform.rescale(em,rescale_factor,anti_aliasing=False)

In [6]:
# This cell will take a few minutes to run.
# If it takes too long, reduce the final_width parameter.

# Make an array with the X and Y locations of the closest gold particles (unit: pixels)
G = np.array(rescale_factor*gold_locations[['YM','XM']] / pixel_size, dtype=int)
# For each pixel, calculate the sum of the distances to the num_closest gold particles 
# D = distance(em_rescaled.shape[0],em_rescaled.shape[1],G,num_closest=num_closest_gold)

In [7]:
# Save results
# logD = np.log(D)
# logD_normalized = (255*((logD-np.min(logD))/(np.max(logD)-np.min(logD)))).astype(np.uint8)
# io.imsave(f'{out_path}/{Path(em_path).stem}-logDistTo{num_closest_gold}ClosestGold.tif',logD_normalized)

# Save image with gold locations

In [8]:
goldLocs = np.zeros(em.shape,dtype=np.uint8)
radius = 10
for ii in range(G.shape[0]):
    rr,cc = draw.disk((G[ii,0],G[ii,1]),radius)
    goldLocs[rr,cc] = 1

io.imsave(f'{out_path}/{Path(em_path).stem}-locGoldParticles.tif',255*goldLocs)

/var/folders/0z/hvg4mzl50gqb0m2ql3c0w0xc0000gn/T/ipykernel_44652/1091997925.py:7: UserWarning: /Users/lukasvandenheuvel/Documents/PhD/Data-temp/20241022_iDA-figures/clem-immunogold/raw/Matek2-cU/ig-detection/Image-0051-locGoldParticles.tif is a low contrast image
  io.imsave(f'{out_path}/{Path(em_path).stem}-locGoldParticles.tif',255*goldLocs)


# View output with napari

In [9]:
import napari

# View heatmap
viewer = napari.Viewer()
viewer.add_image(em_rescaled)
# viewer.add_image(logD_normalized,colormap='inferno_r',blending='additive')
viewer.add_points(G,name='gold_locations',size=50,face_color='red',opacity=0.4)

Assistant skips harvesting pyclesperanto as it's not installed.


<Points layer 'gold_locations' at 0x2b6060eb0>

In [10]:
# View original image with gold detections in red
viewer2 = napari.Viewer()
viewer2.add_image(em)
viewer2.add_points(gold_locations[['YM','XM']] / pixel_size,size=35,edge_width=0,face_color='red',opacity=0.5)

<Points layer 'Points' at 0x2bf716b20>

# Make scalebar

In [11]:
min,max = [np.min(logD)+np.log(pixel_size),np.max(logD)+np.log(pixel_size)] # Min and max value of the distance map (in units of log(nm))
print(f'Min = {min} log(nm), Max = {max} log(nm)')
scalebar = np.repeat(np.arange(0,256).reshape(-1,1), 35, axis=1)
io.imsave(f'{out_path}/{Path(em_path).stem}-logDistTo{num_closest_gold}ClosestGold-scalebar.tif',scalebar.astype(np.uint8))

NameError: name 'logD' is not defined